In [ ]:
import json
import pandas as pd
import plotly.graph_objects as go

# Load JSON config
with open("expenses.json", "r") as f:
    data = json.load(f)

df = pd.DataFrame(data)

# Ensure date is datetime
df["date"] = pd.to_datetime(df["date"])

# Pivot: rows = dates, cols = items, values = sum of prices
pivot = df.pivot_table(index="date", columns="item", values="price", aggfunc="sum", fill_value=0)

# Build stacked bar traces
traces = []
for item in pivot.columns:
    traces.append(
        go.Bar(
            name=item,
            x=pivot.index,
            y=pivot[item],
            text=[f"{v:.2f}" if v > 0 else "" for v in pivot[item]],
            textposition="inside",
            hovertemplate=f"Item: {item}<br>Date: %{x}<br>Price: $%{{y:.2f}}<extra></extra>"
        )
    )

# Daily totals for annotation
day_totals = pivot.sum(axis=1)

total_trace = go.Scatter(
    x=pivot.index,
    y=day_totals,
    mode="text",
    text=[f"${t:.2f}" for t in day_totals],
    textposition="top center",
    showlegend=False,
    hoverinfo="skip"
)

# Layout
layout = go.Layout(
    title="Vacation Spending",
    barmode="stack",
    xaxis=dict(title="Date"),
    yaxis=dict(title="Price (USD)"),
    bargap=0.2
)

fig = go.Figure(data=traces + [total_trace], layout=layout)
fig.show()
# Save to HTML
# fig.write_html("expenses_plot.html", include_plotlyjs="cdn")

# print("Saved plot to expenses_plot.html")


: 